# __Homework 4:__ Practical analysis with BioPython

For the homework, you are going to extend the code from the analysis of our FASTQ file in lectures 8 and 9.
Recall that the FASTQ file contains reads from a real sequencing run of influenza virus HA and NA genes.

---
The __actual sequences__ are as follows:

    5'-[end of HA]-AGGCGGCCGC-[16 X N barcode]-3'
or 

    5'-[end of NA]-AGGCGGCCGC-[16 X N barcode]-3'
---


__The end of NA is__ `...CACGATAGATAAATAATAGTGCACCAT`
    
__The end of HA is__ `...CCGGATTTGCATATAATGATGCACCAT`

---    

    
The __sequencing reads__ from the reverse end of the molecules (in 5'>3' orientation), so the sequencing reads are as follows:

    5'-[reverse complement of 16 X N barcode]-GCGGCCGCCT-[reverse complement of the end of HA]-3'
or

    5'-[reverse complement of 16 X N barcode]-GCGGCCGCCT-[reverse complement of the end of NA]-3'

---   
    
The reads can originate from **either** HA or NA, and that will be distinguished by the most 3' end of the read.
But in our example exercise in class, we did not distinguish among reads matching to HA and NA, as we didn't even look far enough into the read to tell the identity.

For the homework, your goal is to write code that extends the material from lectures 8 and 9 to also distinguish between HA and NA.
This homework can be completed almost entirely by re-using code from lecture 9. You will need to set up your analysis to do the following:
 1. Get the reverse complement of each read.
 2. Determine if it matches the expected pattern for HA and NA, and if so which one.
 3. If it matches, extract the barcode and add it to a dictionary to keep track of counts.
 4. Determine the number and distribution of barcodes for HA and NA separately.

Please include code to address each of the following questions. Please include code comments to explain what your code is attempting to accomplish. Don't forget to include references to the sources you used to obtain your answer, including your classmates (if you are working in groups).

In [13]:
import Bio.SeqIO
import Bio.Seq
import re
reads = Bio.SeqIO.parse('barcodes_R1.fastq', format='fastq')
seqreads = list(reads)
seqreads_Seq = []
for seqrecord in seqreads:
    sequence = seqrecord.seq # isolate the sequence from the seqrecord
    seqreads_Seq.append(sequence) # add string sequence to list

# Sequence list to be used in downstream count_barcodes function
seq_list = []
for sequence in seqreads_Seq:
    temp = str(sequence)
    seq_list.append(temp)
print(seq_list[0:5])

['GCTTAAGTTATTTAGTGCGGCCGCCTATGGTGCACTATTATTTATCTATCGTGAAAGGGAGTTCTGCTCCATCAGGCCAAGATCGGAAGAGCACACGTCTGAACTCCAGTCACATGTCAGAATCTCGTATGCCGTCTTCTGCTTGAAAAAAAAAAAAAAGCTGAAATTAATAATTTTGAAACCAGTTTTGTAAACGCAGCACTAAAATGAAGGCATGCAACGACGATGTTTATTGACACGGAATAGCAGA', 'CTTCCTGGTCACGGTTGCGGCCGCCTATGGTGCATCATTATATGCAAATCCGGCATTGCAAGGAGCCGTTGGAACACATAGATCGGAAGAGCACACGTCTGAACTCCAGTCACATGTCAGAATCTCGTATGCCGTCTTCTGCTTGAAAAAAAAAAAAAAAACCTAAATAGGTATCATAAAGACACCATCAACAGCTTAGTATACAACCCCATGCATTAAACAAATACATCCCTAATGACAGAGTGTACGA', 'CACGGCTATTACCCCGCGGCCGCCTATGGTGCACTATTATTTATCTATCGTGAAAGGGAGTTCTGCTCCATCAGGCCAAGATCGGAAGAGCACACGTCTGAACTCCAGTCACATGTCAGAATCTCGTATGCCGTCTTCTGCTTGAAAAAAAAAACTAACATACAAAATCTAACTCCCACTAGGTATTGAAGCCAATGATCGAACTTATGACACTGATTAACGTAGACGTCCCTTCTACTATACACTACGA', 'TCAGCGATGAATGTGGGCGGCCGCCTATGTTGCATCATTATATGCAAATCCGGCATTGCAAGGAGCCGTTGGAACACATAGATCGGAAGAGCACACGTCTGAACTCCAGTCACATGTCAGAATCTCGTATGCCGTCTTCTGCTTGAAAAAAAAAAAAAAAAGCAAAGCACGCAAAACACAACCATCAGATGAAAGAAGACACTAACCATCCGACCCAACCTCATGCTATCACCAAC

In [11]:
def count_barcodes(seqs, middle='GCGGCCGCCT', Seq_HA='ATGGTGCATCATTATATGCAAATCCGG', Seq_NA='ATGGTGCACTATTATTTATCTATCGTG'):


    # When looking up regular expressions online, I learned you can use both raw strings and f strings in the regular expression library to make the syntax look cleaner 
    # https://stackoverflow.com/questions/21104476/what-does-the-r-in-pythons-re-compiler-pattern-flags-mean
    # https://stackoverflow.com/questions/52980108/how-do-i-use-format-in-re-compile
    
    subtype_pattern_HA = re.compile(f'{Seq_HA}')
    subtype_pattern_NA = re.compile(f'{Seq_NA}')
    # initalize empty lists and dictionaries 
    barcode_dict_HA = {}
    barcode_dict_NA = {}
    no_hit_counter_NA = 0
    no_hit_counter_HA = 0
    total_HA_reads = 0
    total_NA_reads = 0

    for sequence in seqs: 
        query = sequence.upper()
        
        # Check for HA 
        if subtype_pattern_HA.search(query):
            total_HA_reads += 1
            # If HA is found, try to extract barcode
            barcode_pattern = re.compile('(?P<barcode>[ATCG]{16})' + middle + Seq_HA)
            barcode_match = barcode_pattern.search(query)
            # Add barcode to dict 
            if barcode_match:
                barcode = barcode_match.group('barcode')
                if barcode in barcode_dict_HA:
                    barcode_dict_HA[barcode] += 1
                else:
                    barcode_dict_HA[barcode] = 1
            # Document invalid reads
            else:
                no_hit_counter_HA += 1

        # Do the same for NA 
        if subtype_pattern_NA.search(query):
            total_NA_reads += 1
            # If NA is found, try to extract barcode
            barcode_pattern = re.compile('(?P<barcode>[ATCG]{16})' + middle + Seq_NA) 
            barcode_match = barcode_pattern.search(query)
            # Add barcode to dict 
            if barcode_match:
                barcode = barcode_match.group('barcode')
                if barcode in barcode_dict_NA:
                    barcode_dict_NA[barcode] += 1
                else:
                    barcode_dict_NA[barcode] = 1
            # Document invalid reads
            else:
                no_hit_counter_NA += 1

    # using the max function that is described here instead of using a loop to iterate over the values: https://stackoverflow.com/questions/268272/getting-key-with-maximum-value-in-dictionary
    HA_max_count = max(barcode_dict_HA.values())
    HA_max_barcode = max(barcode_dict_HA, key=barcode_dict_HA.get)
    NA_max_count = max(barcode_dict_NA.values())
    NA_max_barcode = max(barcode_dict_NA, key=barcode_dict_NA.get)
    
    

    print("----------------------------------------------------------------------")
    print(barcode_dict_HA)
    print("HA read Count:", total_HA_reads)
    print("HA invalid barcode count:", no_hit_counter_HA)
    print(f'HA barcode with max counts: {HA_max_barcode}, {HA_max_count}')
    print("----------------------------------------------------------------------")
    print(barcode_dict_NA)
    print("NA read Count:", total_NA_reads)
    print("NA invalid barcode count:", no_hit_counter_NA)
    print(f'NA barcode with max counts: {NA_max_barcode}, {NA_max_count}')
    print("----------------------------------------------------------------------")
    
    

1. How many reads map to HA, and how many reads map to NA?

In [12]:
# I chose to create a couple of loops that go through a string list, sort by either HA or NA revcomp ends, and then do some barcode cataloging with regular expression search patterns 
# Because this was all within a function, I chose to make a short summary at the end as an output 
# The number of reads that were first mapped to HA or NA are in the 1st line of the summary via the variables (total_HA_reads, total_NA_reads)
count_barcodes(seq_list)

----------------------------------------------------------------------
{'CTTCCTGGTCACGGTT': 70, 'ATATGGGAGACGATAA': 77, 'AGGGATGACTGGTATG': 28, 'TGACTTATACGTAAGT': 53, 'ATGGTATAGTAGTAGC': 119, 'GCGGACTGTGGGTAAC': 62, 'ACATCCTTGTGTGGTG': 41, 'TAAGTGTGATGTATGA': 47, 'GATTCACGGAGGGTAC': 99, 'CGTGGTAGCGTGGAGT': 31, 'AAGGGTTGAGGAGTGC': 48, 'GGACAGCAGGGAGCGG': 43, 'AGTTATCGCTACGTTT': 61, 'GTAATATGGGACGTGA': 9, 'TTAATGTCGGGTCGGG': 155, 'CGAGTCGACCTCTCGT': 60, 'GTACTAGGAAGTCGAA': 86, 'GTTAGCTAGACTGGGT': 70, 'GCGAAGGCGTATGACC': 16, 'TTGTCGGAAGGTTAAG': 55, 'CTACATTTATTGCCCA': 57, 'TGACACAGGATGAGGG': 54, 'CAGCCTGAATGGAATT': 35, 'GGAGATAAGGCTAGAA': 81, 'AGTTTGAGATTATTCT': 62, 'ACAGACTTGTTTGTTT': 50, 'CCGAGATGTTCGTATT': 49, 'AGATTGCGCAGATTCG': 60, 'AGACTGATCGGAAATC': 124, 'GCTATCAAGACGTTCT': 30, 'TGTTGAAGCACCGGTC': 76, 'TTACGTGTTGTCAGCA': 47, 'CGGTAGGCCCGACTAT': 21, 'CGAGTACCATTGGGAA': 28, 'GGAATTTTCTTGCCAG': 43, 'GTGTACGTTCACCGAA': 76, 'TTGGCGAGTGATGGAG': 115, 'AGCGGCGTGGGGTTGA': 24, 'TTGAAAACGCTA

2. How many HA sequences did not have a valid barcode? Also anwer the same question for NA.

In [175]:
# The "no_hit_counter_NA" and "no_hit_counter_HA" variables were created to store this information 
# They are printed out on the 2nd line of the summary 
count_barcodes(seq_list)

----------------------------------------------------------------------
HA read Count: 5409
HA invalid barcode count: 160
HA barcode with max counts: TTAATGTCGGGTCGGG, 155
----------------------------------------------------------------------
NA read Count: 4122
NA invalid barcode count: 213
NA barcode with max counts: CCCGGGGAGAACTGGT, 152
----------------------------------------------------------------------


3. What is the HA barcode with the most counts (and how many counts)? Also answer the same question for NA.

    _Hint: you will need to find the key associated with the maximum value in your dictionary. There are many ways to do this._

In [174]:
# The variables (HA_max_count, HA_max_barcode, NA_max_count, NA_max_barcode) are used to determine the maximum values in each dictionary 
# I chose to use a built in max function instead of a loop, because the function was getting pretty large and I wanted to try something new
# These values are printed on the 3rd line of the summary 
count_barcodes(seq_list)

----------------------------------------------------------------------
HA read Count: 5409
HA invalid barcode count: 160
HA barcode with max counts: TTAATGTCGGGTCGGG, 155
----------------------------------------------------------------------
NA read Count: 4122
NA invalid barcode count: 213
NA barcode with max counts: CCCGGGGAGAACTGGT, 152
----------------------------------------------------------------------
